In [ ]:
import requests
import numpy as np
!pip install "tensorflow>=1.14.0,<2.0"
!pip install "tensorflow-gpu>=1.14.0,<2.0"
!pip install  ampligraph
import functools
import itertools
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_hub as hub

from tqdm import trange

In [2]:
import ampligraph
import numpy as np
ampligraph.__version__
import pandas as pd
import csv
from ampligraph.datasets import load_from_ntriples
from ampligraph.datasets import load_from_csv

In [3]:
Y_train=load_from_csv('/content','1300train.csv',sep=',',add_reciprocal_rels=False)

In [4]:
Y_test=load_from_csv('/content','1000test.csv',sep=',',add_reciprocal_rels=False)

In [ ]:
Y_train.shape

In [6]:
entities = np.unique(np.concatenate([Y_train[:, 0], Y_train[:, 2]]))
entities.size

2090

In [7]:
Y_test.shape

(25, 3)

In [8]:
from ampligraph.latent_features import DistMult

In [9]:
model = DistMult(batches_count=10, 
                seed=0, 
                epochs=20, 
                k=10, 
                eta=10,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [10]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(Y_train, early_stopping = False)

Average Loss:   2.374241: 100%|██████████| 20/20 [00:00<00:00, 32.83epoch/s]


In [11]:
relations = np.unique(Y_train[:, 1])
relations.size

553

In [19]:
df = pd.read_csv("embeddings_projector (5).tsv", sep='\t')

In [ ]:

hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(df)

In [12]:
import pandas as pd
from sklearn import preprocessing

from statistics import mean
data = pd.read_csv('1300train.csv',header=None)
data.head()
arr =[]

type(data)

for i,j,k in zip(data[0],data[1],data[2]):
  print(i,j,k)
  H1=model.get_embeddings([i],embedding_type='entity')
  T1=model.get_embeddings([k],embedding_type='entity')
  R1=model.get_embeddings([j], embedding_type='relation')
  df3=np.add(H1,R1)-T1
  normalized = preprocessing.normalize(df3)
  # print("Normalized Data = ", normalized)
  ar=abs(normalized)
  # print(ar)
  # break
  for i in ar:
    Score=mean(i)
    # print(Score)
    arr.append(Score)

print(arr)




Common symptoms include fever
Management involves treatment
The World Health Organization declared outbreak
common symptoms include cough
minority develop noticeable symptoms
Cardiovascular complications include heart failure
Sputum carry large amounts
indicative suggest underlying immunopathology
people have classical serum biomarkers
The US Food and Drug Administration approved test
guidelines recommend medication
Face coverings limit volume
the United States Environmental Protection Agency maintains list
Intensivists compiled treatment recommendations
availability affect mortality
study reported earliest date
Official publications reported earliest onset
Wuhan Central Hospital sent bronchoalveolar lavage fluid BAL sample
the National Health Commission of China issued notice
China reported nearly 140 new cases
Italy overtook China
the United States overtaken China
ratio reflects number
the World Health Organization reported estimates
random antibody testing suggested national IFR
Ear

In [13]:
import pandas
df = pandas.DataFrame(data={"col1": arr})
df.to_csv("./scores.csv", sep=',',index=False)

In [14]:
from ampligraph.evaluation import evaluate_performance

In [15]:
positives_filter =np.concatenate((Y_test,Y_train))

In [ ]:
from ampligraph.utils import save_model, restore_model

In [ ]:
save_model(model, './covid3_model.pkl')

In [16]:
ranks = evaluate_performance(Y_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 25/25 [00:00<00:00, 284.46it/s]


In [18]:
df = pd.DataFrame(data=ranks, columns=["Subject(corrupt)", "Object(corrupt)"])

In [19]:
df.to_csv('Test_triple_ranks.csv')

In [17]:
print(ranks)

[[ 883  851]
 [   1    1]
 [ 830  746]
 [ 791  885]
 [1750 1498]
 [ 874  857]
 [ 532  169]
 [1307 1374]
 [1929 1764]
 [1160 1198]
 [1606 1670]
 [1719 1617]
 [ 227  152]
 [ 927  937]
 [ 320  402]
 [ 276  249]
 [ 839  828]
 [1764 1471]
 [ 225  301]
 [1623 1370]
 [1950 2087]
 [  61    1]
 [1997 1840]
 [ 417  528]
 [1421 1483]]


In [20]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
mr = mr_score(ranks)
mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))
hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.06
MR: 994.16
Hits@10: 0.06
Hits@3: 0.06
Hits@1: 0.06


In [21]:
from ampligraph.utils import create_tensorboard_visualizations

In [22]:
create_tensorboard_visualizations(model,r"./")

In [35]:
X_unseen = np.array([
    ['guidelines', 'recommend', 'medication'],
  ['black musician', 'wears', 'royal livery'],
   ['Common effects', 'include', 'chest infections'],
  # ['pandemic', 'include', 'global social economic disruption']

])

In [36]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))

In [37]:
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_unseen=True,
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)

100%|██████████| 3/3 [00:00<00:00, 48.85it/s]


In [38]:
scores = model.predict(X_unseen)

In [39]:
from scipy.special import expit
probs = expit(scores)

In [40]:
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")

,statement,rank,score,prob
1,black musician wears royal livery,96,0.003997,0.500999
2,Common effects include chest infections,1158,0.005087,0.501272
0,guidelines recommend medication,1,0.024346,0.506086
